In [63]:
!pip install gudhi==3.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [146]:
import gudhi
import numpy as np

In [227]:
# Datos originales

data = {
    0: [3,3,2,2,3,1,1,2,2,2,2,4,4.7,4,4,4.7,4.5],
    1: [4,4,5,4,4,3,4,3,4,5,4,4,4.8,2,4.3,4.4,4.9],
    2: [2,2,1,1,2,1,2,3,2,2,1,2,4.8,2,3.6,4.5,4.8],
    3: [2,1,1,2,3,3,3,2,3,2,2,3,4.9,3,4.1,4.9,4.7],
    4: [2,2,2,3,3,2,2,3,3,2,3,2,4.3,3,3.0,3.0,4.2],
    5: [5,3,3,3,5,4,4,3,5,2,4,2,5.0,1,4.8,4.7,None],
    6: [2,1,3,2,2,2,3,2,2,2,2,1,4.4,2,4.1,4.1,3.8],
    7: [5,3,5,3,4,5,4,3,2,5,3,2,4.3,2,4.9,4.4,4.2],
    8: [2,2,1,4,2,5,2,2,3,1,3,3,4.6,2,4.4,None,3.9],
    9: [3,2,2,3,4,3,4,3,2,2,2,3,4.0,3,4.7,3.2,3.0],
    10: [1,2,2,3,1,3,2,1,2,2,3,3,4.7,2,4.4,4,4.2],
    11: [4,1,2,1,1,1,1,1,2,2,2,1,3.7,4,3.8,4.4,3.7],
    12: [3,4,2,3,4,1,2,3,2,2,4,3,4.3,3,4.7,4.8,4.2],
    13: [1,2,1,1,2,1,3,3,1,1,1,1,3.2,1,4,3.5,3.7],
    14: [1,2,2,2,2,1,1,1,2,2,2,3,4.5,2,4.0,3.4,4.1],
    15: [2,1,2,4,3,1,2,2,2,2,3,2,4.7,3,3.3,4.3,4.1]
}


# Rellenar valores faltantes
def fill_missing_with_mean(data):
    names = list(data.keys())
    matrix = np.array(list(data.values()), dtype=float)
    col_means = np.nanmean(np.where(matrix == None, np.nan, matrix), axis=0)
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i, j] is None or np.isnan(matrix[i, j]):
                matrix[i, j] = col_means[j]
    
    filled_data = {names[i]: matrix[i].tolist() for i in range(len(names))}
    
    return filled_data

# Corrección de notas
data_filled = fill_missing_with_mean(data)
for datas in data_filled:
    for datos in [-1, -2, -3, -5]:
          data_filled[datas][datos] = 5-data_filled[datas][datos]
        

# Estandarización de los datos
def transponer_matriz(A):
    return [list(fila) for fila in zip(*A)]
def estand(A):
    keys = list(A.keys())
    a = [A[k] for k in keys]
    a_t = transponer_matriz(a)
    for columna in a_t:
        mini = min(columna)
        maxi = max(columna)
        if maxi - mini != 0:
            for i in range(len(columna)):
                columna[i] = (columna[i] - mini) / (maxi - mini)
        else:
            for i in range(len(columna)):
                columna[i] = 0
    a_std = transponer_matriz(a_t)
    return a_std

points = estand(data_filled)

In [229]:
# Matriz de distancia triangular
def Distance_Matrix(distance, points):
    matrix = []
    for i in range(len(points)):
        row = []
        for j in range(i):
            row.append(distance(points[i], points[j]))
        matrix.append(row)

    return matrix


# Distancia Euclidea, se puede cambiar a otras
def distance(a, b):
    res = 0
    for i in range(len(a)):
        res += (a[i] - b[i])**2
    return res**0.5

# Máxima distancia para el complejo simplicial
Matrix = Distance_Matrix(distance, points)

for row in Matrix:
    print(row)

distances = []
for row in Matrix:
    for el in row:
        distances.append(el)

distances.sort()

n = len(distances)
epsilon = (distances[n//2] + distances[n//2+1])/2

# Esto es para hacer el complejo
rips_complex = gudhi.RipsComplex(
    distance_matrix = Matrix, 
    max_edge_length = epsilon
)

# Esto es una estructura para que le de las caras del complejo
simplex_tree = rips_complex.create_simplex_tree(max_dimension=16)

fmt = '%s -> %.2f'
count = 0
print(epsilon)
for filtered_value in simplex_tree.get_filtration():
    count += 1
print(count)

[]
[2.131129674534681]
[1.361709279934127, 2.4570526188017383]
[1.259320268258196, 2.1223151001577447, 1.1878958610604138]
[1.6091537499075799, 2.1102468018992377, 1.4662452643632318, 1.564000895022332]
[2.2759185672416526, 1.4524550290109512, 2.270802477946437, 1.8789672528128458, 2.082459547959132]
[1.6879982991476963, 2.3012113878962652, 1.2482067092659217, 1.213209833146852, 1.349790549256797, 1.97438096863173]
[2.2632348650344065, 1.285649771639559, 2.3382134328125774, 2.0437467026995595, 2.141168595593376, 1.3792711006883092, 1.89103482378402]
[1.6958026031938414, 2.117212696027225, 1.8378937748360584, 1.2838683514997808, 1.5157336328245778, 1.786496012822374, 1.483427337960526, 2.0103713084363135]
[1.8816084822538728, 2.0737565871670296, 1.9353511596995778, 1.6685815407896856, 1.4679818664526882, 1.844340980607397, 1.4584775097647267, 1.7047247543665809, 1.545947551486884]
[1.4849195768069352, 2.2154309122019242, 1.6693851585433244, 1.2545759408479844, 1.5902985683856838, 2.1875

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0869eb2d-21ed-4b31-a4f6-d09a7c1322ce' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>